In [ ]:
# Model selection
API_KEY = "your-openai-api-key-here"
MODEL = "grok-3"

In [ ]:
# Model configuration
import openai
from rich.console import Console
import time
from openai import OpenAIError  

console = Console()

def chat_completion(messages, max_retries=3, backoff_factor=2):
    for attempt in range(max_retries):
        try:
            client = openai.OpenAI(
                api_key=API_KEY,
                base_url="https://api.x.ai/v1"
            )
            response = client.chat.completions.create(
                model=MODEL,
                messages=messages,
                temperature=0,
                max_tokens=2048
            )
            return response.choices[0].message.content
        except OpenAIError as e:  
            if e.status_code == 429:  
                sleep_time = backoff_factor ** attempt  
                console.print(f"[bold yellow]Rate limit hit (429). Retrying after {sleep_time} seconds... (Attempt {attempt+1}/{max_retries})[/]")
                time.sleep(sleep_time)
            else:
                console.print(f"[bold red]API request failed: {str(e)}[/]")
                return None
    console.print(f"[bold red]Max retries reached for API call.[/]")
    return None

# Function to read PMIDs from a text file (统一使用此版本)
def read_pmid_from_txt(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            pmids = [line.strip() for line in file if line.strip()]
        return pmids
    except FileNotFoundError:
        console.print(f"[bold red]Error: File {file_path} not found.[/]")
        return []
    except Exception as e:
        console.print(f"[bold red]Error reading {file_path}: {str(e)}[/]")
        return []

In [ ]:
# =====================================
# PubMed元数据抓取模块
# =====================================
# 功能: 从PubMed页面抓取文章元数据（标题、作者、DOI、关键词等）
# 注意: PMC全文抓取请使用下面的优化版本 (v2.1)
# =====================================

import re
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Setup the web driver for scraping
def setup_driver():
    """配置Selenium WebDriver用于PubMed页面抓取"""
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/91.0.4472.124")
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument('--no-sandbox')
    driver = webdriver.Chrome(options=chrome_options)
    return driver

# Fetch data from PubMed using the provided PMID
def fetch_pubmed_data(pmid, driver):
    """
    从PubMed页面抓取文章元数据
    
    Args:
        pmid: PubMed文章ID
        driver: Selenium WebDriver实例
    
    Returns:
        dict: 包含标题、作者、DOI、关键词、期刊名、PMCID等元数据
    """
    url = f"https://www.ncbi.nlm.nih.gov/pubmed/{pmid}"
    driver.get(url)
    
    data = {'pmid': pmid}
    wait = WebDriverWait(driver, 5)
    
    try:
        data['title'] = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'heading-title'))).text.strip()
    except:
        data['title'] = None

    try:
        authors = driver.find_element(By.CLASS_NAME, 'authors-list').text
        data['authors'] = ', '.join(re.sub(r'\s*\d+', '', authors).split(','))
    except:
        data['authors'] = None

    try:
        affiliation = driver.find_element(By.CLASS_NAME, 'affiliation-link').get_attribute('title')
        data['first_author_last_affiliation_word'] = affiliation.split(',')[-1].strip().split()[-1].rstrip('.')
    except:
        data['first_author_last_affiliation_word'] = None

    try:
        data['doi'] = driver.find_element(By.XPATH, '//a[@data-ga-action="DOI"]').text.strip()
    except:
        data['doi'] = None

    try:
        keywords = driver.find_element(By.XPATH, '//p[strong[contains(text(),"Keywords")]]').text
        data['keywords'] = keywords.replace("Keywords:", "").strip().rstrip('.')
    except:
        data['keywords'] = None

    try:
        data['journal_name'] = driver.find_element(By.XPATH, '//meta[@name="citation_publisher"]').get_attribute('content').strip()
    except:
        data['journal_name'] = None

    try:
        pmcid_element = driver.find_element(By.XPATH, '//a[contains(@href, "pmc.ncbi.nlm.nih.gov/articles/PMC")]')
        pmcid_full = pmcid_element.text.strip()
        data['pmcid'] = pmcid_full.replace("PMCID: ", "").strip()
    except:
        data['pmcid'] = None

    return data


In [ ]:
# =====================================
# 🔧 优化后的PMC全文抓取模块 (v2.1)
# =====================================
# 功能改进:
# 1. 精确提取正文内容，排除References/Footnotes等非正文章节
# 2. 三层fallback机制，提高抓取成功率
# 3. 完善的错误处理和日志记录
# 4. 专门适配 https://pmc.ncbi.nlm.nih.gov/articles/PMC{ID}/ 页面结构
# 5. 支持main-article-body精确定位
# =====================================

import re
import time
import requests
from bs4 import BeautifulSoup
from rich.console import Console
from dataclasses import dataclass
from typing import Optional, List, Dict, Tuple
from enum import Enum
import traceback

# 可选: Selenium支持
try:
    from selenium import webdriver
    from selenium.webdriver.chrome.options import Options
    from selenium.webdriver.common.by import By
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    SELENIUM_AVAILABLE = True
except ImportError:
    SELENIUM_AVAILABLE = False
    print("⚠ Selenium未安装，将仅使用requests抓取")

console = Console()

# ============== 状态与结果类 ==============

class ExtractionStatus(Enum):
    """抓取状态枚举"""
    SUCCESS = "success"
    PARTIAL = "partial"      # 部分成功（使用了fallback）
    FAILED = "failed"
    NO_PMCID = "no_pmcid"

@dataclass
class ExtractionResult:
    """抓取结果数据类"""
    pmcid: str
    status: ExtractionStatus
    full_text: Optional[str] = None
    full_text_chunks: Optional[List[str]] = None
    error_message: Optional[str] = None
    method_used: Optional[str] = None
    sections_found: Optional[List[str]] = None
    char_count: int = 0
    word_count: int = 0

# ============== 配置常量 ==============

# 需要排除的章节标题（到达这些章节时停止抓取）
EXCLUDE_SECTION_PATTERNS = [
    r'^references?$',
    r'^bibliography$',
    r'^acknowledgm?ents?$',
    r'^acknowledg?ments?$',
    r'^author\s*contributions?$',
    r'^authors?\s*contributions?$',
    r'^contributors?$',
    r'^contributor\s*information$',
    r'^conflicts?\s*of\s*interests?$',
    r'^competing\s*interests?$',
    r'^declarations?$',
    r'^disclosure$',
    r'^funding$',
    r'^funding\s*(sources?|information)?$',
    r'^financial\s*disclosure$',
    r'^supplementary\s*(materials?|data|information)?$',
    r'^supporting\s*information$',
    r'^appendi(x|ces)$',
    r'^data\s*availability',
    r'^ethics\s*(statement|approval)?$',
    r'^ethical\s*approval$',
    r'^footnotes?$',
    r'^abbreviations?$',
    r'^associated\s*data$',
]

# 正文章节标题（用于识别正文开始位置）
MAIN_CONTENT_PATTERNS = [
    r'^abstract$',
    r'^highlights?$',
    r'^background$',
    r'^introduction$',
    r'^methods?$',
    r'^materials?\s*(and|&)\s*methods?$',
    r'^patients?\s*(and|&)\s*methods?$',
    r'^study\s*design$',
    r'^results?$',
    r'^findings?$',
    r'^discussion$',
    r'^conclusions?$',
    r'^summary$',
]

# ============== 工具函数 ==============

def is_exclude_section(title: str) -> bool:
    """检查章节标题是否应该被排除"""
    if not title:
        return False
    title_clean = title.strip().lower()
    for pattern in EXCLUDE_SECTION_PATTERNS:
        if re.match(pattern, title_clean, re.IGNORECASE):
            return True
    return False

def is_main_content_section(title: str) -> bool:
    """检查章节标题是否为正文章节"""
    if not title:
        return False
    title_clean = title.strip().lower()
    for pattern in MAIN_CONTENT_PATTERNS:
        if re.match(pattern, title_clean, re.IGNORECASE):
            return True
    return False

def clean_text_v2(text: str) -> str:
    """清洗文本：去除多余空白、ORCID、邮箱等"""
    if not text:
        return ""
    
    # 去除ORCID ID
    text = re.sub(r'ORCID\s*(ID)?\s*:\s*\S+', '', text, flags=re.IGNORECASE)
    text = re.sub(r'https?://orcid\.org/\S+', '', text)
    
    # 去除邮箱地址
    text = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', '', text)
    
    # 去除Figure/Table标注中的重复内容
    text = re.sub(r'(Figure|Table)\s*\d+\.?\s*(Open in a new tab)?', r'\1 ', text, flags=re.I)
    
    # 去除多余空白
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'\n\s*\n', '\n\n', text)
    
    return text.strip()

def chunk_text_v2(text: str, chunk_size_words: int = 4000, overlap_words: int = 1000) -> List[str]:
    """将文本分块，带重叠"""
    words = text.split()
    if len(words) <= chunk_size_words:
        return [text]
    
    chunks = []
    start = 0
    while start < len(words):
        end = min(start + chunk_size_words, len(words))
        chunk = ' '.join(words[start:end])
        chunks.append(chunk)
        if end == len(words):
            break
        start += chunk_size_words - overlap_words
    return chunks

# ============== PMC抓取器类 ==============

class PMCScraperV2:
    """
    PMC全文抓取器 v2.1
    
    专门适配PMC网页结构，支持多种抓取策略：
    - 方法1: 基于main-article-body的精确提取（推荐）
    - 方法2: 基于标题标签定位的提取
    - 方法3: Fallback全文提取后正则截断
    
    使用示例:
        scraper = PMCScraperV2()
        result = scraper.extract_full_text("PMC12372713")
        print(result.full_text)
    """
    
    def __init__(self, use_selenium: bool = False, timeout: int = 30):
        self.use_selenium = use_selenium and SELENIUM_AVAILABLE
        self.timeout = timeout
        self.driver = None
        self.request_headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
            'Accept-Language': 'en-US,en;q=0.5',
            'Accept-Encoding': 'gzip, deflate, br',
            'Connection': 'keep-alive',
        }
        
    def setup_driver(self):
        """配置Selenium WebDriver"""
        if not SELENIUM_AVAILABLE:
            raise RuntimeError("Selenium未安装")
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        chrome_options.add_argument(f"user-agent={self.request_headers['User-Agent']}")
        chrome_options.add_argument('--disable-gpu')
        chrome_options.add_argument('--no-sandbox')
        chrome_options.add_argument('--disable-dev-shm-usage')
        chrome_options.add_argument('--disable-blink-features=AutomationControlled')
        self.driver = webdriver.Chrome(options=chrome_options)
        return self.driver
    
    def close_driver(self):
        """关闭WebDriver"""
        if self.driver:
            try:
                self.driver.quit()
            except:
                pass
            self.driver = None
    
    def fetch_html(self, pmcid: str) -> Tuple[Optional[str], Optional[str]]:
        """
        获取PMC页面HTML
        
        Returns:
            Tuple[html_content, error_message]
        """
        url = f"https://pmc.ncbi.nlm.nih.gov/articles/{pmcid}/"
        
        # 方法1: 使用requests（更快、更稳定）
        try:
            response = requests.get(url, headers=self.request_headers, timeout=self.timeout)
            response.raise_for_status()
            
            # 检查是否获取到有效内容（PMC页面通常>50KB）
            if len(response.text) > 10000:
                console.print(f"[green]  ✓ requests成功 ({len(response.text)//1024}KB)[/]")
                return response.text, None
            else:
                console.print(f"[yellow]  ⚠ 页面内容过短 ({len(response.text)}B)[/]")
        except requests.Timeout:
            console.print(f"[yellow]  ⚠ requests超时，尝试Selenium...[/]")
        except requests.HTTPError as e:
            if e.response.status_code == 404:
                return None, f"文章不存在 (404): {pmcid}"
            console.print(f"[yellow]  ⚠ HTTP错误 {e.response.status_code}[/]")
        except requests.RequestException as e:
            console.print(f"[yellow]  ⚠ requests失败: {str(e)[:50]}[/]")
        
        # 方法2: Selenium fallback（处理JS渲染页面）
        if self.use_selenium:
            try:
                console.print("[cyan]  尝试Selenium抓取...[/]")
                if not self.driver:
                    self.setup_driver()
                self.driver.get(url)
                WebDriverWait(self.driver, self.timeout).until(
                    EC.presence_of_element_located((By.CLASS_NAME, 'main-article-body'))
                )
                time.sleep(1)  # 等待内容完全加载
                html = self.driver.page_source
                if len(html) > 10000:
                    console.print(f"[green]  ✓ Selenium成功 ({len(html)//1024}KB)[/]")
                    return html, None
            except Exception as e:
                console.print(f"[red]  ✗ Selenium失败: {str(e)[:50]}[/]")
                return None, f"Selenium失败: {str(e)}"
        
        return None, "无法获取页面内容"
    
    def extract_method1_main_body(self, soup: BeautifulSoup) -> Tuple[Optional[str], List[str]]:
        """
        方法1: 基于main-article-body的精确提取（推荐）
        
        PMC页面结构:
        <section class="body main-article-body">
            <section class="abstract" id="abstract1">...</section>
            <section id="sec1">...</section>  # Methods
            <section id="sec2">...</section>  # Results
            ...
        </section>
        """
        sections_found = []
        content_parts = []
        
        # 查找主文章体
        main_body = soup.find('section', class_='main-article-body')
        if not main_body:
            # Fallback: 尝试其他可能的容器
            main_body = soup.find('div', class_='article-content') or \
                        soup.find('article', class_='article')
        
        if not main_body:
            return None, ["未找到main-article-body"]
        
        console.print(f"[dim]    找到主文章容器[/]")
        
        # 遍历主体内的所有section
        for section in main_body.find_all('section', recursive=False):
            # 获取章节标题
            heading = section.find(['h2', 'h3', 'h4'])
            section_title = heading.get_text(strip=True) if heading else ""
            
            # 检查是否应该停止
            if section_title and is_exclude_section(section_title):
                console.print(f"[dim]    ⏹ 停止于: {section_title}[/]")
                break
            
            # 提取章节内容
            section_text = section.get_text(separator=' ', strip=True)
            if section_text and len(section_text) > 50:
                content_parts.append(section_text)
                if section_title:
                    sections_found.append(section_title)
                    console.print(f"[dim]    ✓ 提取: {section_title[:40]}... ({len(section_text)}字符)[/]")
        
        if content_parts:
            return ' '.join(content_parts), sections_found
        return None, sections_found
    
    def extract_method2_heading(self, soup: BeautifulSoup) -> Tuple[Optional[str], List[str]]:
        """
        方法2: 基于标题标签定位的提取
        
        从Abstract/Introduction开始，到References/Footnotes结束
        """
        sections_found = []
        content_parts = []
        headings = soup.find_all(['h2', 'h3'])
        start_found = False
        
        for heading in headings:
            heading_text = heading.get_text(strip=True)
            
            # 检查是否到达排除章节
            if is_exclude_section(heading_text):
                console.print(f"[dim]    ⏹ 停止于: {heading_text}[/]")
                break
            
            # 检查是否到达正文开始
            if is_main_content_section(heading_text):
                start_found = True
            
            if start_found:
                sections_found.append(heading_text)
                
                # 提取该标题到下一个标题之间的内容
                content = []
                current = heading.find_next_sibling()
                while current:
                    if current.name in ['h2', 'h3']:
                        break
                    text = current.get_text(separator=' ', strip=True)
                    if text:
                        content.append(text)
                    current = current.find_next_sibling()
                
                if content:
                    content_parts.append(f"{heading_text}\n{' '.join(content)}")
        
        if content_parts:
            return '\n\n'.join(content_parts), sections_found
        return None, sections_found
    
    def extract_method3_fallback(self, soup: BeautifulSoup) -> Tuple[Optional[str], List[str]]:
        """
        方法3: Fallback - 整体提取后使用正则截断
        
        适用于非标准页面结构
        """
        sections_found = ["[Fallback模式]"]
        
        # 尝试多种容器选择器
        article = soup.find('article') or \
                  soup.find('div', class_='article-content') or \
                  soup.find('div', id=re.compile(r'article|content', re.I)) or \
                  soup.find('main')
        
        if not article:
            return None, sections_found
        
        # 移除无用元素
        for tag in article.find_all(['script', 'style', 'nav', 'footer', 'header', 'aside']):
            tag.decompose()
        
        # 移除所有排除章节及其内容
        for heading in article.find_all(['h1', 'h2', 'h3', 'h4']):
            if is_exclude_section(heading.get_text(strip=True)):
                # 移除该标题及所有后续兄弟元素
                for sibling in list(heading.find_next_siblings()):
                    sibling.decompose()
                heading.decompose()
                break
        
        text = article.get_text(separator=' ', strip=True)
        
        # 再次用正则确保没有References内容
        for pattern in [r'\bReferences\b', r'\bBibliography\b', r'\bFootnotes\b']:
            match = re.search(pattern, text, re.IGNORECASE)
            if match:
                text = text[:match.start()]
                break
        
        return (text.strip(), sections_found) if len(text) > 500 else (None, sections_found)
    
    def extract_full_text(self, pmcid: str) -> ExtractionResult:
        """
        主提取函数：依次尝试多种方法提取全文
        
        Args:
            pmcid: PMC文章ID，如 "PMC12372713" 或 "12372713"
        
        Returns:
            ExtractionResult: 包含提取结果的数据类
        """
        # 参数验证
        if not pmcid:
            return ExtractionResult(
                pmcid="", 
                status=ExtractionStatus.NO_PMCID, 
                error_message="PMCID为空"
            )
        
        # 标准化PMCID格式
        pmcid = pmcid.strip()
        if not pmcid.upper().startswith('PMC'):
            pmcid = f"PMC{pmcid}"
        pmcid = pmcid.upper()
        
        console.print(f"\n[bold blue]📄 抓取 {pmcid}...[/]")
        
        try:
            # 获取HTML
            html, error = self.fetch_html(pmcid)
            if not html:
                return ExtractionResult(
                    pmcid=pmcid, 
                    status=ExtractionStatus.FAILED, 
                    error_message=error or "无法获取HTML"
                )
            
            soup = BeautifulSoup(html, 'html.parser')
            
            # 方法1: main-article-body精确提取（推荐）
            console.print("[cyan]  方法1: main-article-body精确提取[/]")
            text, sections = self.extract_method1_main_body(soup)
            if text and len(text) > 1000:
                clean = clean_text_v2(text)
                console.print(f"[green]  ✓ 方法1成功 ({len(sections)}章节, {len(clean)}字符)[/]")
                return ExtractionResult(
                    pmcid=pmcid, 
                    status=ExtractionStatus.SUCCESS,
                    full_text=clean, 
                    full_text_chunks=chunk_text_v2(clean),
                    method_used="method1_main_body", 
                    sections_found=sections,
                    char_count=len(clean),
                    word_count=len(clean.split())
                )
            
            # 方法2: 标题定位提取
            console.print("[cyan]  方法2: 标题定位提取[/]")
            text, sections = self.extract_method2_heading(soup)
            if text and len(text) > 1000:
                clean = clean_text_v2(text)
                console.print(f"[green]  ✓ 方法2成功 ({len(sections)}章节, {len(clean)}字符)[/]")
                return ExtractionResult(
                    pmcid=pmcid, 
                    status=ExtractionStatus.SUCCESS,
                    full_text=clean, 
                    full_text_chunks=chunk_text_v2(clean),
                    method_used="method2_heading", 
                    sections_found=sections,
                    char_count=len(clean),
                    word_count=len(clean.split())
                )
            
            # 方法3: Fallback
            console.print("[yellow]  方法3: Fallback提取[/]")
            text, sections = self.extract_method3_fallback(soup)
            if text and len(text) > 500:
                clean = clean_text_v2(text)
                console.print(f"[yellow]  ⚠ Fallback成功 ({len(clean)}字符)[/]")
                return ExtractionResult(
                    pmcid=pmcid, 
                    status=ExtractionStatus.PARTIAL,
                    full_text=clean, 
                    full_text_chunks=chunk_text_v2(clean),
                    method_used="method3_fallback", 
                    sections_found=sections,
                    char_count=len(clean),
                    word_count=len(clean.split())
                )
            
            # 所有方法都失败
            return ExtractionResult(
                pmcid=pmcid, 
                status=ExtractionStatus.FAILED, 
                error_message="所有提取方法均失败，页面结构可能不标准"
            )
            
        except Exception as e:
            console.print(f"[red]  ✗ 异常: {str(e)}[/]")
            return ExtractionResult(
                pmcid=pmcid, 
                status=ExtractionStatus.FAILED, 
                error_message=f"异常: {str(e)}\n{traceback.format_exc()[:500]}"
            )

# ============== 便捷函数 ==============

def fetch_pmc_full_text_v2(pmcid: str, driver=None) -> Dict:
    """
    便捷函数：获取PMC全文（兼容原有接口）
    
    Args:
        pmcid: PMC文章ID
        driver: 可选的Selenium driver（未使用，保留兼容性）
    
    Returns:
        Dict: 包含 pmcid, full_text, full_text_chunks, status, error 等字段
    """
    scraper = PMCScraperV2(use_selenium=False)
    result = scraper.extract_full_text(pmcid)
    
    return {
        'pmcid': result.pmcid,
        'full_text': result.full_text,
        'full_text_chunks': result.full_text_chunks,
        'status': result.status.value,
        'error': result.error_message,
        'method': result.method_used,
        'sections': result.sections_found,
        'char_count': result.char_count,
        'word_count': result.word_count
    }

# ============== 测试函数 ==============

def test_pmc_scraper(pmcid: str = "PMC12372713"):
    """
    测试抓取效果
    
    使用示例:
        test_pmc_scraper("PMC12372713")
        test_pmc_scraper("12372713")  # 也可以不带PMC前缀
    """
    console.print(f"\n[bold magenta]{'='*60}[/]")
    console.print(f"[bold magenta]🧪 测试PMC抓取器 v2.1 - {pmcid}[/]")
    console.print(f"[bold magenta]{'='*60}[/]")
    
    scraper = PMCScraperV2(use_selenium=False)
    result = scraper.extract_full_text(pmcid)
    
    console.print(f"\n[bold]📊 抓取结果:[/]")
    console.print(f"  PMCID:  {result.pmcid}")
    console.print(f"  状态:   {result.status.value}")
    console.print(f"  方法:   {result.method_used}")
    console.print(f"  章节:   {result.sections_found}")
    
    if result.full_text:
        console.print(f"  字符数: {result.char_count:,}")
        console.print(f"  词数:   {result.word_count:,}")
        console.print(f"  分块数: {len(result.full_text_chunks)}")
        
        console.print(f"\n[bold]📝 文本预览 (前800字符):[/]")
        console.print("-" * 60)
        console.print(result.full_text[:800] + "...")
        console.print("-" * 60)
        
        console.print(f"\n[bold]📝 文本末尾 (后500字符):[/]")
        console.print("-" * 60)
        console.print("..." + result.full_text[-500:])
        console.print("-" * 60)
        
        # 验证是否包含References
        tail = result.full_text.lower()[-1000:]
        if any(kw in tail for kw in ['references', 'bibliography', 'footnotes']):
            console.print("[red]⚠ 警告: 文本末尾可能包含非正文内容![/]")
        else:
            console.print("[green]✓ 验证通过: 未检测到References/Footnotes等内容[/]")
    else:
        console.print(f"\n[red]  错误: {result.error_message}[/]")
    
    return result

# ============== 直接运行测试 ==============
# 取消下面的注释来运行测试
# test_pmc_scraper("PMC12372713")

In [ ]:
# Define prompts used for extracting data

def get_prompts():
    prompt1 = """
You are a medical research assistant specializing in the study of clinical risk prediction models. You assist users in collecting information and providing answers by identifying and analyzing medical information, statistical data, and prediction models within text. 
[Extracted text]
Please analyze the above text and extract detailed information related to the study design and its key parameters. Specifically, extract the following elements.
1. Study Type: Indicate whether the data used in the research comes from a prospective study (cohort study) or a retrospective study (case-control study).
2. Disease Name: Specify the target disease studied in the text.
3. Data Sources: Extract details about the primary data source, such as the name of the dataset, or the specific institution or public database from which the data were obtained.
4. External Validation: Determine whether the study includes an external validation set, which refers to a dataset originating from a different source than the model development set. All subsequent references to the validation set refer specifically to external validation, not internal test sets.
5. Date Range: Specify the dates of the collected participant data for model development and model external validation, including the start and end dates.
6. Follow-up Time: Specify the follow-up time of the cohort for the model development stage, if applicable.
7. Sample Characteristics: Indicate whether the included samples had a specific disease history, underwent particular surgeries or treatments, or were in a certain condition (e.g., pregnancy, smoking).
8. Sample Sizes: Extract the sample sizes for the development set (including training set and test set) and external validation set, if mentioned.
9. Case and Control Numbers: What are the numbers of cases and controls in the development set? What are the numbers of cases and controls in the external validation set? Perform inference and calculation as much as possible based on the provided sample data.
10. Participant Statistics: Extract demographic information about participants in the development and external validation stages, including: the number of female participants; the number of male participants. Perform inference and calculation as much as possible based on the provided sample data.
11. Extract age information about participants in the development and external validation stages, including the age range of participants; the average age and standard deviation of participants; the median age and interquartile range of participants.
12. The racial or ethnic composition of participants in the development and external validation stages.
    """
    prompt2 = """
Continue extracting the following details related to the prediction models from the research. If the paper constructs or evaluates multiple models, please extract the following information for all models.
1. Model Type: Identify the type of prediction models used (e.g., logistic regression, random forests, deep learning).
2. Prediction Variables: List the variables used in every prediction model.
3. Report the AUC values, including confidence intervals, for the development stage and the validation stage.
4. Report the C-index values, including confidence intervals, for the development stage and the validation stage.
5. Report accuracy values, including confidence intervals, for the development stage and the validation stage.
6. Report F1 scores, including confidence intervals, for the development stage and the validation stage.
7. Determine whether calibration values are reported for the development stage and the validation stage, such as calibration curve, Hosmer–Lemeshow test, Brier Score, or expected vs. observed outcomes. If available, please report them.
8. Nomogram Development: Indicate whether a nomogram was constructed based on the model.
9. Use of TRIPOD Guidelines: State whether the study followed the TRIPOD guidelines, if applicable.
    """
    prompt3 = """
Please format the extracted information into two columns: one column for the field and one column for the value, ensuring that each field corresponds to only one value. Use the following fields to create the table. This table consists of 25 fields.
1. Study Type: e.g., prospective study, retrospective study. If it is not mentioned in the text, answer NA.
2. Disease Name: e.g. lung cancer.
3. External Validation: Answer Yes or No.
4. Date Range in the Development Set: e.g., 1992–2008. If it is not mentioned in the text, answer NA.
5. Date Range in the Validation Set: e.g., 1992–2008. If this study lacks external validation, answer N/A.
6. Median Follow-up Time (Years) and IQR: e.g., 2.95 years (IQR: 1.71–4.83). If it is not mentioned in the text, answer NA.
7. Mean Follow-up Time (Years) and Standard Error: e.g., 2.95 years (SD: 0.71). If it is not mentioned in the text, answer NA.
8. Data Sources: e.g., the Kaiser Permanente Washington Breast Cancer Surveillance Consortium (BCSC) registry, the GEO database, the Cancer Screening Program in Urban China (CanSPUC) conducted in Henan Province. If it is not mentioned in the text, answer NA.
9. Sample Characteristics: e.g., patients with a history of diabetes, pregnant women, ever-smokers. If it is not mentioned in the text, answer N/A.
10. Number of Cases in the Development Set: e.g., 5,323. If it is not mentioned in the text, answer NA.
11. Number of Controls in the Development Set: e.g., 5,323. If it is not mentioned in the text, answer NA.
12. Number of Cases in the External Validation Set: e.g., 5,323. If this study lacks external validation, answer N/A.
13. Number of Controls in the External Validation Set: e.g., 5,323. If this study lacks external validation, answer N/A.
14. Number of Female Participants (Development): e.g., 86. If it is not mentioned in the text, answer NA.
15. Number of Female Participants (External Validation): e.g., 86. If this study lacks external validation, answer N/A.
16. Number of Male Participants (Development): e.g., 545. If it is not mentioned in the text, answer NA.
17. Number of Male Participants (External Validation): e.g., 545. If this study lacks external validation, answer N/A.
18. Age Range (Development): e.g., 22–60. If it is not mentioned in the text, answer NA.
19. Age Range (External Validation): e.g., 22–60. If this study lacks external validation, answer N/A.
20. Average Age and Standard Deviation (Development): e.g., 74.2 years (SD = 8.2). If it is not mentioned in the text, answer NA.
21. Average Age and Standard Deviation (External Validation): e.g., 74.2 years (SD = 8.2). If this study lacks external validation, answer N/A.
22. Median Age (Development) and IQR: e.g., 43.2 years (IQR: 32.43–64.83). If it is not mentioned in the text, answer NA.
23. Median Age (External Validation) and IQR: e.g., 43.2 years (IQR: 32.43–64.83). If this study lacks external validation, answer N/A.
24. Racial/Ethnic Composition (Development): e.g., White: 80.4%, Asian: 8.8%, Black: 3.9%, Other or multiple races: 5.3%, Unknown: 1.5%. If it is not mentioned in the text, answer NA.
25. Racial/Ethnic Composition (External Validation): e.g., White: 80.4%, Asian: 8.8%, Black: 3.9%, Other or multiple races: 5.3%, Unknown: 1.5%. If this study lacks external validation, answer N/A.
    """
    prompt4 = """
Please extract and format the following information for the models in the study. Organize the data into a two-column or multi-column table, with one column for the field name (e.g., Model Name, Model Type, etc.) and the remaining columns for values. Each column corresponds to one model's values. If there are multiple models, a multi-column table is needed. Each field should occupy only one row in the table. This table consists of 10 fields.
1. Model Name: e.g., Tyrer-Cuzick Model, LiFeCRC Score.
2. Model Type: e.g., logistic regression, random forest, support vector machine.
3. Prediction Variables: e.g., age, gender, education, smoking status, blood pressure medication, prevalent stroke.
4. AUC Values: e.g., 0.767 (95% CI: 0.749–0.786). If a validation stage is present, prioritize reporting the values from the validation stage.
5. C-index Values: e.g., 0.767 (95% CI: 0.749–0.786). If a validation stage is present, prioritize reporting the values from the validation stage.
6. Accuracy: e.g., 74.8% (95% CI: 71.30% - 78.30%). If a validation stage is present, prioritize reporting the values from the validation stage.
7. F1-score: e.g., 74.8% (95% CI: 71.30% - 78.30%). If a validation stage is present, prioritize reporting the values from the validation stage.
8. Calibration Values: e.g., Hosmer-Lemeshow (HL) test p-value = 0.428; O/E ratio ranged from 0.79 to 1.22, or a brief description. If a validation stage is present, prioritize reporting the values from the validation stage.
9. Nomogram Application: Answer Yes or No.
10. Use of TRIPOD Guidelines: Answer Yes or No.
    """
    return [prompt1, prompt2, prompt3, prompt4]

In [ ]:
# Structured data storage
import re
from prompts import get_prompts
from utils import chat_completion
from rich.console import Console

console = Console()

# Define the fields for sample and model data
sample_fields = [
    "Study Type",
    "Disease Name",
    "External Validation",
    "Date Range in the Development Set",
    "Date Range in the Validation Set",
    "Median Follow-up Time (Years) and IQR",
    "Mean Follow-up Time (Years) and Standard Error",
    "Data Sources",
    "Sample Characteristics",
    "Number of Cases in the Development Set",
    "Number of Controls in the Development Set",
    "Number of Cases in the Validation Set",
    "Number of Controls in the Validation Set",
    "Number of Female Participants (Development)",
    "Number of Female Participants (Validation)",
    "Number of Male Participants (Development)",
    "Number of Male Participants (Validation)",
    "Age Range (Development)",
    "Age Range (Validation)",
    "Average Age and Standard Deviation (Development)",
    "Average Age and Standard Deviation (Validation)",
    "Median Age (Development) and IQR",
    "Median Age (Validation) and IQR",
    "Racial/Ethnic Composition (Development)",
    "Racial/Ethnic Composition (Validation)"
]

model_fields = [
    "Model Name",
    "Model Type",
    "Prediction Variables",
    "AUC Values",
    "C-index Values",
    "Accuracy",
    "F1-score",
    "Calibration Values",
    "Nomogram Application",
    "Use of TRIPOD Guidelines"
]

# Process input text using the prompts and obtain responses from Grok-3
def process_input(input_text, prompts):
    responses = []
    for i, prompt in enumerate(prompts, 1):
        messages = [{"role": "user", "content": prompt.replace("[Extracted text]", input_text)}]
        console.print(f"[italic yellow]Processing Prompt {i}...[/]")
        response = chat_completion(messages)
        if response:
            responses.append(response)
            console.print(f"\n[bold magenta]Response {i}:[/]")
            console.print(response)
            console.print("\n" + "-"*50 + "\n")
        else:
            console.print(f"[bold red]Prompt {i} processing failed[/]")
    return responses

# Extract sample data from the response based on predefined fields
def extract_sample_data(response_sample, pmid):
    data_dict = {field: "" for field in sample_fields}
    data_dict["PMID"] = pmid
    lines_sample = response_sample.splitlines()
    
    for line in lines_sample:
        match = re.match(r'\s*\d+\.\s+([^0-9].*?)\s{2,}(.*)', line.strip())
        if match:
            field, value = match.groups()
            for sample_field in sample_fields:
                if sample_field in field:
                    data_dict[sample_field] = value
                    break

    return data_dict

# Extract model data from the response based on predefined fields, handling multiple models
def extract_model_data(response_model, pmid):
    model_data_list = []
    lines_model = response_model.splitlines()
    
    current_model = {field: "" for field in model_fields}
    current_model["PMID"] = pmid
    
    for line in lines_model:
        match = re.match(r'\s*\d+\.\s+([^0-9].*?)\s{2,}(.*)', line.strip())
        if match:
            field, value = match.groups()
            if "Model Name" in field and current_model["Model Name"]:
                # New model detected, save the current one and start a new one
                model_data_list.append(current_model)
                current_model = {field: "" for field in model_fields}
                current_model["PMID"] = pmid
            for model_field in model_fields:
                if model_field in field:
                    current_model[model_field] = value
                    break
    
    # Append the last model
    if current_model["Model Name"] or any(current_model[field] for field in model_fields if field != "PMID"):
        model_data_list.append(current_model)
    
    # If no models were identified, return a single empty model with PMID
    if not model_data_list:
        model_data_list.append({field: "" for field in model_fields})
        model_data_list[0]["PMID"] = pmid
    
    return model_data_list

In [ ]:
# main progrom

import csv
from rich.console import Console

# Configuration
console = Console()

def main():
    console.print("[bold green]Welcome to the Clinical Risk Prediction Model Analysis Tool[/]")
    txt_file = 'PMID.TXT'
    pmids = read_pmid_from_txt(txt_file)
    if not pmids:
        console.print("[bold red]Failed to read PMID[/]")
        return

    with open('literature_data.csv', 'a', newline='', encoding='utf-8') as literature_csv, \
         open('sample_data.csv', 'a', newline='', encoding='utf-8') as sample_csv, \
         open('model_information.csv', 'a', newline='', encoding='utf-8') as model_csv:
        
        literature_writer = csv.DictWriter(literature_csv, fieldnames=literature_fields)
        sample_writer = csv.DictWriter(sample_csv, fieldnames=sample_fields)
        model_writer = csv.DictWriter(model_csv, fieldnames=model_fields)
        
        if literature_csv.tell() == 0:
            literature_writer.writeheader()
        if sample_csv.tell() == 0:
            sample_writer.writeheader()
        if model_csv.tell() == 0:
            model_writer.writeheader()

        for pmid in pmids:
            console.print(f"[italic yellow]Processing PMID: {pmid}...[/]")
            driver = setup_driver()
            pubmed_data = fetch_pubmed_data(pmid, driver)
            full_text_data = fetch_pmc_full_text(pubmed_data['pmcid'], driver)
            full_text = full_text_data.get('full_text', "")
            
            literature_writer.writerow({
                'PMID': pmid,
                'Title': pubmed_data.get('title', ''),
                'Authors': pubmed_data.get('authors', ''),
                'First Author Last Affiliation Word': pubmed_data.get('first_author_last_affiliation_word', ''),
                'DOI': pubmed_data.get('doi', ''),
                'Keywords': pubmed_data.get('keywords', ''),
                'Journal Name': pubmed_data.get('journal_name', ''),
                'PMCID': pubmed_data.get('pmcid', ''),
                'Full Text': full_text
            })
            
            prompts = get_prompts()
            prompt1_with_fulltext = prompts[0].replace("[input text]", full_text)
            responses = process_input(prompt1_with_fulltext, prompts)
            
            if len(responses) >= 4:
                response_sample = responses[2]  # Prompt 3
                response_model = responses[3]  # Prompt 4
                
                sample_data = extract_sample_data(response_sample, pmid)
                model_data = extract_model_data(response_model, pmid)
                
                sample_writer.writerow(sample_data)
                model_writer.writerow(model_data)
            
            driver.quit()
            print(f"Information related to PMID {pmid} has been saved.")

if __name__ == "__main__":
    main()

In [ ]:
# Double-LLM-Validation

import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import xml.etree.ElementTree as ET

def get_pmcid_from_pmid(pmid):
    try:
        url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id={pmid}&rettype=xml"
        response = requests.get(url)
        response.raise_for_status()
        root = ET.fromstring(response.content)
        pmcid_elem = root.find('.//PMID[@Version="1"]/..//ArticleId[@IdType="pmc"]')
        if pmcid_elem is not None:
            return pmcid_elem.text
        else:
            return None
    except Exception as e:
        print(f"Error fetching PMCID for PMID {pmid}: {str(e)}")
        return None

def disagreement_resolution(model, model_key, agreement_filepath):
    df = pd.read_excel(agreement_filepath)

    variables = df["Variable"]
    gr = df["GROK_Responses"] 
    cr = df["CLAUDE_Responses"]
    ad = df["Agree(A)/Disagree(D)"]
    pmids = df["PMID"]

    allresponses_list = []
    count = 0

    for i in range(len(cr)):
        if ad[i] == "D":  # Disagreement Check
            pmid = pmids[i] 
            pmcid = get_pmcid_from_pmid(pmid)
            if pmcid is None:
                print(f"PMCID not found for PMID {pmid}, skipping.")
                continue

            print(f"Processing PMID {pmid} \t Count: {count}\n")
            count += 1

            # Prompt Design
            var = str(variables[i])
            if model == "claude-sonnet-4-20250514":
                val = str(gr[i])  # GROK_Responses
            else:
                val = str(cr[i])
            p = "For the given text, LLM generate the response = [" + val + "] for the variable = [" + var + "]. Verify if the response for the given variable generated by LLM is correct or incorrect. If the response is incorrect, then generate the correct response (as short and precise as possible)."

            # Acquiring text
            try:
                url = f"https://pmc.ncbi.nlm.nih.gov/articles/{pmcid}/"
                response = requests.get(url)
                response.raise_for_status()  # Check status
                soup = BeautifulSoup(response.text, 'html.parser')
                abstract_section = soup.find("section", {"class": "abstract", "id": "abstract1"})
                references_section = soup.find("h2", {"class": "pmc_sec_title"}, string="References")
                if abstract_section and references_section:
                    content = ""
                    current = abstract_section.find_next()
                    while current and current != references_section:
                        if current.name == "section":
                            content += current.get_text(separator=" ", strip=True) + "\n"
                        current = current.find_next()
                else:
                    content = "Failed to extract content between Abstract and References."
                print(f"Extracted Content Length: {len(content)} characters\n")
            except Exception as e:
                content = f"Error fetching content for PMCID {pmcid}: {str(e)}"
                print(content)

            allresponses = "PMID: " + str(pmid) + "\n"
            allresponses += "PMCID: " + pmcid + "\n"
            allresponses += "Variable Name: " + var + "\n"
            allresponses += "GROK Response: " + str(gr[i]) + "\n"  # GROK Response
            allresponses += "Claude Response: " + str(cr[i]) + "\n\n"
            allresponses += "Verification from the Text...\n"

            # Processing text
            if model == "claude":
                response = claude_function(content, p, model_key, model)
                response = str(response)
                time.sleep(30)  # Bypass TPM restrictions
            else:
                response = gpt_function(content, p, model_key, model)
            allresponses = allresponses + "\n" + response

            print("\n")
            allresponses_list.append(allresponses)

    ff = "DisagreementResolution_by_" + model
    write_raw_responses(allresponses_list, ff)

a = input("Do you to execute Disagreement Resolution Module? Y/N")
if a == "Y" or a == "y":
    model = input("Enter the Model Name")
    key = input("Enter the Key")
    agreement_filepath = input("Enter the Path to the Annotated Agreement Matching File")
    disagreement_resolution(model, key, agreement_filepath)
else:
    print("You Choose Not to Execute Agreement/Disagreement Module.")